# GAT Event Embedding
This notebook establishes a training pipeline for our Event Embedding model.

## Installing our libraries and required scripts

In [ ]:
!git clone https://github.com/joaopedromattos/pyGAT
!pip install --quiet spektral

Cloning into 'pyGAT'...
remote: Enumerating objects: 213, done.
remote: Total 213 (delta 0), reused 0 (delta 0), pack-reused 213
Receiving objects: 100% (213/213), 324.96 KiB | 1.89 MiB/s, done.
Resolving deltas: 100% (117/117), done.
     |████████████████████████████████| 122kB 7.4MB/s 


In [ ]:
!pip install xlwt

In [ ]:
!pip install git+https://github.com/rmarcacini/sentence-transformers
!pip install gdown
!gdown https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
!gdown https://drive.google.com/uc?id=1A4_X5MgY4N6uaE_nM1XvVdxRhRq1X9VV
!unzip distiluse-base-multilingual-cased.zip -d language_model
!mkdir generated_graphs && mv large_generated_graphs.zip ./generated_graphs && unzip ./generated_graphs/large_generated_graphs.zip -d ./generated_graphs
from sentence_transformers import SentenceTransformer, LoggingHandler
language_model = SentenceTransformer('distiluse-base-multilingual-cased')

  Cloning https://github.com/rmarcacini/sentence-transformers to /tmp/pip-req-build-htz660q3
  Running command git clone -q https://github.com/rmarcacini/sentence-transformers /tmp/pip-req-build-htz660q3
     |████████████████████████████████| 890kB 8.1MB/s 
     |████████████████████████████████| 1.2MB 19.9MB/s 
     |████████████████████████████████| 3.0MB 49.1MB/s 
     |████████████████████████████████| 901kB 45.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.6-cp37-none-any.whl size=101897 sha256=a366755715361c5b2fb5856f6feb262b87651ea81b065fc1b3566b5915d6c163
  Stored in directory: /tmp/pip-ephem-wheel-cache-dlczgy79/wheels/88/3c/66/55ee9fb698480d5a5116a8257c15dc363323e4922fb8ad361b
Successfully built sentence-transformers
Downloading...
From: https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
To: /content/distiluse-base-multilingual-cased.zip
504MB [00:09, 52.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1A4_X5MgY4N

100%|██████████| 504M/504M [00:29<00:00, 16.9MB/s]


In [ ]:
import os

os.chdir('./pyGAT')

In [ ]:
import networkx as nx
from tqdm import tqdm
import pandas as pd

import numpy as np
import logging
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, LabelEncoder
from event_graph_utils import mount_graph, regularization, process_event_dataset_from_networkx
from train import GAT_wrapper

# auth.authenticate_user()
# print('Authenticated')

## 5W1H Graph Events

In [ ]:
!gdown --id 1RF_bIo5ndxPhu9SJw-T8HBcuHyaGQGL0

Downloading...
From: https://drive.google.com/uc?id=1RF_bIo5ndxPhu9SJw-T8HBcuHyaGQGL0
To: /content/pyGAT/datasets.tar.gz
22.7MB [00:00, 27.9MB/s]


In [ ]:
!tar -xzvf datasets.tar.gz

datasets_runs/
datasets_runs/run_1_google_news_5w1h_graph_hin.nx
datasets_runs/run_6_40er_5w1h_graph_hin.nx
datasets_runs/run_4_bbc_5w1h_graph_hin.nx
datasets_runs/run_8_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_5_bbc_5w1h_graph_hin.nx
datasets_runs/run_9_google_news_5w1h_graph_hin.nx
datasets_runs/run_5_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_2_bbc_5w1h_graph_hin.nx
datasets_runs/run_9_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_7_40er_5w1h_graph_hin.nx
datasets_runs/run_9_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_8_google_news_5w1h_graph_hin.nx
datasets_runs/run_10_bbc_5w1h_graph_hin.nx
datasets_runs/run_8_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_2_news_cluster_5w1h_graph_hin.nx
datasets_runs/run_8_40er_5w1h_graph_hin.nx
datasets_runs/run_6_bbc_5w1h_graph_hin.nx
datasets_runs/run_4_google_news_5w1h_graph_hin.nx
datasets_runs/run_2_google_news_5w1h_graph_hin.nx
datasets_runs/run_7_gold_standard_5w1h_graph_hin.nx
datasets_runs/run_4_gold_standard_

# Regularization

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import random


def regularization(G, dim, embedding_feature: str = 'embedding', iterations=15, mi=0.85):

    nodes = []

    # inicializando vetor f para todos os nodes
    for node in G.nodes():
        G.nodes[node]['f'] = np.array([0.0]*dim)
        if embedding_feature in G.nodes[node]:
            G.nodes[node]['f'] = G.nodes[node][embedding_feature]*1.0
        nodes.append(node)

    pbar = tqdm(range(0, iterations))

    for iteration in pbar:
        random.shuffle(nodes)
        energy = 0.0

        # percorrendo cada node
        for node in nodes:
            f_new = np.array([0.0]*dim)
            f_old = np.array(G.nodes[node]['f'])*1.0
            sum_w = 0.0

            # percorrendo vizinhos do onde
            for neighbor in G.neighbors(node):
                w = 1.0
                if 'weight' in G[node][neighbor]:
                    w = G[node][neighbor]['weight']

                w /= np.sqrt(G.degree[neighbor])

                f_new += w*G.nodes[neighbor]['f']

                sum_w += w

            f_new /= sum_w

            G.nodes[node]['f'] = f_new*1.0

            if embedding_feature in G.nodes[node]:
                G.nodes[node]['f'] = G.nodes[node][embedding_feature] * \
                    mi + G.nodes[node]['f']*(1.0-mi)

            energy += np.linalg.norm(f_new-f_old)

        iteration += 1
        message = 'Iteration '+str(iteration)+' | Energy = '+str(energy)
        pbar.set_description(message)

    return G

In [ ]:
import numpy as np
import networkx as nx
import random
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import logging
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split


def process_event_dataset_from_networkx(G, features_attr="f"):
    """
    Builds an event graph dataset used in GAT model
    Parameters:
        G -> Graph representation of the event network (Networkx graph)
        df_labels -> user labeled data
        features_att -> Feature attribute of each node (str)
        random_state -> A random seed to train_test_split
    Returns:
        adj -> Sparse and symmetric adjacency matrix of our graph.
        features -> A NumPy matrix with our graph features.
        idx_train -> A NumPy array with the indexes of the training nodes.
        idx_val -> A NumPy array with the indexes of the validation nodes.
        idx_test -> A NumPy array with the indexes of the test nodes.
    """

    num_nodes = len(G.nodes)
    print("Num nodes", num_nodes)


    

    # validation_split_percentage = val_split / (1 - train_split)

    # df_val, df_test = train_test_split(
    #     df_test_and_val, train_size=validation_split_percentage, random_state=random_state)

    # Organizing our feature matrix...
    # feature_matrix = np.array([ G.nodes[i]['embedding'] if 'embedding' in G.nodes[i].keys() else G.nodes[i][features_attr] for i in G.nodes()])
    #features = np.array([G.nodes[i][features_attr] for i in G.nodes()])
    L_features = []
    L_train = []
    L_test = []
    L_labels = []
    label_codes = {}
    for node in G.nodes():
      L_features.append( (G.nodes[node]['id'], G.nodes[node]['f']) )
      if 'train' in G.nodes[node]: L_train.append(G.nodes[node]['id'])
      if 'test' in G.nodes[node]: L_test.append(G.nodes[node]['id'])
      if 'label' in G.nodes[node]:
        if G.nodes[node]['label'] not in label_codes: label_codes[G.nodes[node]['label']] = len(label_codes) 
        L_labels.append( [G.nodes[node]['id'],G.nodes[node]['label'],label_codes[G.nodes[node]['label']]] )
    df_features = pd.DataFrame(L_features)
    df_features.columns = ['node_id','embedding']
    features = np.array(df_features.sort_values(by=['node_id'])['embedding'].to_list())

    idx_train = L_train
    idx_test = L_test
    labels = [-1]*num_nodes
    df_labels = pd.DataFrame(L_labels)
    print(df_labels)
    df_labels.columns = ['event_id','label','label_code']

    
    for index,row in df_labels.iterrows():
        labels[int(row['event_id'])] = row['label_code']

    adj = nx.adjacency_matrix(G)

    return adj, features, labels, idx_train, idx_test, df_labels

## Training

In [ ]:
from __future__ import division
from __future__ import print_function

import os
import glob
import time
import random
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import scipy.sparse as sp


from models import GAT, SpGAT


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

class Namespace(object):
    def __init__(self, adict):
        self.__dict__.update(adict)

class GAT_wrapper_v2():
    def __init__(self, args={"alpha": 0.2, "cuda": True, "dropout": 0.6, "epochs": 10, "fastmode": False, "hidden": 8, "lr": 0.005, "nb_heads": 8, "no_cuda": False, "patience": 100, "seed": 72, "sparse": False, "weight_decay": 0.0005}):

        if (type(args) == dict):
            args = Namespace(args)

        self.args = args

        self.model = None

        self.loss_test = 0.0
        self.acc_test = 0.0

        self.adj = None
        self.features = None
        self.labels = None
        self.idx_train = None
        self.idx_val = None
        self.idx_test = None

    def compute_test(self):
        self.model.eval()
        output = self.model(self.features, self.adj)
        loss_test = F.nll_loss(
            output[self.idx_test], self.labels[self.idx_test])
        acc_test = accuracy(output[self.idx_test], self.labels[self.idx_test])
        print("Test set results:",
              "loss= {:.4f}".format(loss_test.item()),
              "accuracy= {:.4f}".format(acc_test.item()))

        self.loss_test = loss_test
        self.acc_test = acc_test

        return loss_test, acc_test, output[self.idx_test].max(1)[1]

    def train_pipeline(self, adj, features, labels, idx_train, idx_val, idx_test, *args):

        adj = normalize_adj(adj + sp.eye(adj.shape[0]))

        if (sp.issparse(adj)):
            adj = adj.todense()

        if (sp.issparse(features)):
            features = features.todense()

        # With networkx, we no longer need to convert from one-hot encoding...
        #labels = np.where(labels)[1]

        adj = torch.FloatTensor(adj)
        features = torch.FloatTensor(features)
        labels = torch.LongTensor(labels)
        idx_train = torch.LongTensor(idx_train)
        idx_val = torch.LongTensor(idx_val)
        idx_test = torch.LongTensor(idx_test)

        random.seed(self.args.seed)
        np.random.seed(self.args.seed)
        torch.manual_seed(self.args.seed)
        if self.args.cuda:
            torch.cuda.manual_seed(self.args.seed)

        # Load data
        # adj, features, labels, idx_train, idx_val, idx_test = new_load_data(
        #     *args, custom_function=custom_function, function=function)

        # Model and optimizer
        if self.args.sparse:
            model = SpGAT(nfeat=features.shape[1],
                          nhid=self.args.hidden,
                          nclass=int(labels.max()) + 1,
                          dropout=self.args.dropout,
                          nheads=self.args.nb_heads,
                          alpha=self.args.alpha)
        else:
            model = GAT(nfeat=features.shape[1],
                        nhid=self.args.hidden,
                        nclass=int(labels.max()) + 1,
                        dropout=self.args.dropout,
                        nheads=self.args.nb_heads,
                        alpha=self.args.alpha)
        optimizer = optim.Adam(model.parameters(),
                               lr=self.args.lr,
                               weight_decay=self.args.weight_decay)

        if self.args.cuda:
            model.cuda()
            features = features.cuda()
            adj = adj.cuda()
            labels = labels.cuda()
            idx_train = idx_train.cuda()
            idx_val = idx_val.cuda()
            idx_test = idx_test.cuda()

        features, adj, labels = Variable(
            features), Variable(adj), Variable(labels)

        # TODO: Test if these lines could be written below line 41.
        self.adj = adj
        self.features = features
        self.labels = labels
        self.idx_train = idx_train
        self.idx_val = idx_val
        self.idx_test = idx_test

        def train(epoch):
            t = time.time()
            model.train()
            optimizer.zero_grad()
            output = model(features, adj)
            loss_train = F.nll_loss(output[idx_train], labels[idx_train])
            acc_train = accuracy(output[idx_train], labels[idx_train])
            loss_train.backward()
            optimizer.step()

            if not self.args.fastmode:
                # Evaluate validation set performance separately,
                # deactivates dropout during validation run.
                model.eval()
                output = model(features, adj)

            loss_val = F.nll_loss(output[idx_val], labels[idx_val])
            acc_val = accuracy(output[idx_val], labels[idx_val])
            print('Epoch: {:04d}'.format(epoch+1),
                  'loss_train: {:.4f}'.format(loss_train.data.item()),
                  'acc_train: {:.4f}'.format(acc_train.data.item()),
                  'loss_val: {:.4f}'.format(loss_val.data.item()),
                  'acc_val: {:.4f}'.format(acc_val.data.item()),
                  'time: {:.4f}s'.format(time.time() - t))

            return loss_val.data.item()

        # Train model
        t_total = time.time()
        loss_values = []
        bad_counter = 0
        best = self.args.epochs + 1
        best_epoch = 0
        for epoch in range(self.args.epochs):
            loss_values.append(train(epoch))

            torch.save(model.state_dict(), '{}.pkl'.format(epoch))
            if loss_values[-1] < best:
                best = loss_values[-1]
                best_epoch = epoch
                bad_counter = 0
            else:
                bad_counter += 1

            if bad_counter == self.args.patience:
                break

            files = glob.glob('*.pkl')
            for file in files:
                epoch_nb = int(file.split('.')[0])
                if epoch_nb < best_epoch:
                    os.remove(file)

        files = glob.glob('*.pkl')
        for file in files:
            epoch_nb = int(file.split('.')[0])
            if epoch_nb > best_epoch:
                os.remove(file)

        print("Optimization Finished!")
        print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

        # Restore best model
        print('Loading {}th epoch'.format(best_epoch))
        model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

        self.model = model

        return model



In [ ]:
from os import listdir
from os.path import isfile, join
path_datasets = '../generated_graphs/'
network_files = [f for f in listdir(path_datasets) if isfile(join(path_datasets, f)) and ('.edge_list' in f)]
print(network_files)

['GraphRNN_RNN_EVENT_4_128_pred_10_1_1000_nodes.edge_list', 'GraphRNN_RNN_EVENT_4_128_pred_10_1_4000_nodes.edge_list', 'GraphRNN_RNN_EVENT_4_128_pred_10_1_3000_nodes.edge_list', 'GraphRNN_RNN_EVENT_4_128_pred_10_1_1500_nodes.edge_list', 'GraphRNN_RNN_EVENT_4_128_pred_10_1_2000_nodes.edge_list', 'GraphRNN_RNN_EVENT_4_128_pred_10_1_3500_nodes.edge_list', 'GraphRNN_RNN_EVENT_4_128_pred_10_1_2500_nodes.edge_list']


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm
import time

for i in range(1, 6):

    experimental_results = []

    for network_file in tqdm(network_files):

        print('Networkfile', network_file)

        G = nx.read_edgelist(path_datasets+network_file)
        print("OK...", G)
        
        
        for k in G.nodes:
            G.nodes[k]['features'] = np.random.rand(512)
            G.nodes[k]['id'] = int(k)
            G.nodes[k]['label'] = np.random.choice(['event', 'location', 'time'])
            G.nodes[k][np.random.choice(['train', 'test'], p=[0.8, 0.2])] = True

        time_start = time.time()
        regularization(G, 512, embedding_feature='features')
        adj, features, labels, idx_train, idx_test, df_labels = process_event_dataset_from_networkx(
            G)
        print(adj.shape, features.shape, len(idx_train), len(idx_test))
        gat = GAT_wrapper_v2({"alpha": 0.2, "cuda": True, "dropout": 0.5, "epochs": 50, "fastmode": True, "hidden": 8,
                            "lr": 0.005, "nb_heads": 8, "no_cuda": False, "patience": 100, "seed": 72, "sparse": True, "weight_decay": 0.0005})
        gat.train_pipeline(adj, features, labels, idx_train, idx_train, idx_test)
        loss, acc, output = gat.compute_test()
        time_end = time.time()
        
        if output.is_cuda:
            y_pred = output.cpu().numpy()    
        else:
            y_pred = output.numpy()
        y_true = []
        for event_id in idx_test:
            for node in G.nodes():
                # if ':event' in node:
                if G.nodes[node]['id'] == event_id:
                    y_true.append(
                        df_labels[df_labels.event_id == event_id].label_code.values[0])
        f1_macro = f1_score(y_true, y_pred, average='macro')
        acc = accuracy_score(y_true, y_pred)

        print('--->', network_file, 'f1_macro', f1_macro, 'acc', acc)
        experimental_results.append(
            (network_file, 'f1_macro', f1_macro, 'acc', acc, y_true, y_pred, 'time', time_end - time_start))
        del gat
        del adj
        del features
        del G

    df_results = pd.DataFrame(experimental_results)
    df_results.to_csv(f'./gat_results_r{i}_news_cluster_5w1h_graph_hin.csv', index=False, sep=';')
    df_results.to_excel(f'./gat_results_r{i}_news_cluster_5w1h_graph_hin.xls')

Networkfile GraphRNN_RNN_EVENT_4_128_pred_10_1_1000_nodes.edge_list
OK... 



Num nodes 1001
        0         1  2
0       0  location  0
1       1      time  1
2       2  location  0
3       3      time  1
4       4     event  2
...   ...       ... ..
996   990  location  0
997   999  location  0
998   998  location  0
999   989  location  0
1000  983      time  1

[1001 rows x 3 columns]
(1001, 1001) (1001, 512) 792 209
Epoch: 0001 loss_train: 1.6305 acc_train: 0.3169 loss_val: 1.6305 acc_val: 0.3169 time: 0.0401s
Epoch: 0002 loss_train: 2.6142 acc_train: 0.3308 loss_val: 2.6142 acc_val: 0.3308 time: 0.0377s
Epoch: 0003 loss_train: 3.1221 acc_train: 0.3220 loss_val: 3.1221 acc_val: 0.3220 time: 0.0349s
Epoch: 0004 loss_train: 1.8544 acc_train: 0.3232 loss_val: 1.8544 acc_val: 0.3232 time: 0.0349s
Epoch: 0005 loss_train: 1.2813 acc_train: 0.3270 loss_val: 1.2813 acc_val: 0.3270 time: 0.0337s
Epoch: 0006 loss_train: 1.1394 acc_train: 0.3699 loss_val: 1.1394 acc_val: 0.3699 time: 0.0339s
Epoch: 0007 loss_train: 1.1211 acc_train: 0.3687 loss_val: 1.1211 acc_val:


Num nodes 1526
         0         1  2
0        0  location  0
1        1      time  1
2        2  location  0
3        4      time  1
4        5     event  2
...    ...       ... ..
1521  1513     event  2
1522  1522  location  0
1523  1516      time  1
1524  1517     event  2
1525  1521      time  1

[1526 rows x 3 columns]
(1526, 1526) (1526, 512) 1205 321
Epoch: 0001 loss_train: 2.3584 acc_train: 0.3477 loss_val: 2.3584 acc_val: 0.3477 time: 0.0354s
Epoch: 0002 loss_train: 1.3963 acc_train: 0.3328 loss_val: 1.3963 acc_val: 0.3328 time: 0.0370s
Epoch: 0003 loss_train: 1.1517 acc_train: 0.3145 loss_val: 1.1517 acc_val: 0.3145 time: 0.0361s
Epoch: 0004 loss_train: 1.1183 acc_train: 0.3369 loss_val: 1.1183 acc_val: 0.3369 time: 0.0351s
Epoch: 0005 loss_train: 1.1162 acc_train: 0.3195 loss_val: 1.1162 acc_val: 0.3195 time: 0.0348s
Epoch: 0006 loss_train: 1.1011 acc_train: 0.3154 loss_val: 1.1011 acc_val: 0.3154 time: 0.0320s
Epoch: 0007 loss_train: 1.1101 acc_train: 0.3220 loss_val: 1.


Num nodes 3001
         0         1  2
0        0  location  0
1        1      time  1
2        2  location  0
3        3      time  1
4        4     event  2
...    ...       ... ..
2996  2995     event  2
2997  2982  location  0
2998  2989  location  0
2999  2996  location  0
3000  3000  location  0

[3001 rows x 3 columns]
(3001, 3001) (3001, 512) 2373 628
Epoch: 0001 loss_train: 1.5838 acc_train: 0.3064 loss_val: 1.5838 acc_val: 0.3064 time: 0.0672s
Epoch: 0002 loss_train: 5.4149 acc_train: 0.3253 loss_val: 5.4149 acc_val: 0.3253 time: 0.0605s
Epoch: 0003 loss_train: 2.1769 acc_train: 0.3464 loss_val: 2.1769 acc_val: 0.3464 time: 0.0547s
Epoch: 0004 loss_train: 1.5455 acc_train: 0.3595 loss_val: 1.5455 acc_val: 0.3595 time: 0.0491s
Epoch: 0005 loss_train: 1.1177 acc_train: 0.3316 loss_val: 1.1177 acc_val: 0.3316 time: 0.0488s
Epoch: 0006 loss_train: 1.1046 acc_train: 0.3262 loss_val: 1.1046 acc_val: 0.3262 time: 0.0480s
Epoch: 0007 loss_train: 1.1190 acc_train: 0.3177 loss_val: 1.


Num nodes 1501
         0         1  2
0        0  location  0
1        1      time  1
2        2  location  0
3        3      time  1
4        7     event  2
...    ...       ... ..
1496  1493  location  0
1497  1494      time  1
1498  1489      time  1
1499  1499     event  2
1500  1497     event  2

[1501 rows x 3 columns]
(1501, 1501) (1501, 512) 1187 314
Epoch: 0001 loss_train: 1.6830 acc_train: 0.3184 loss_val: 1.6830 acc_val: 0.3184 time: 0.0423s
Epoch: 0002 loss_train: 5.8103 acc_train: 0.3243 loss_val: 5.8103 acc_val: 0.3243 time: 0.0371s
Epoch: 0003 loss_train: 1.4574 acc_train: 0.3142 loss_val: 1.4574 acc_val: 0.3142 time: 0.0401s
Epoch: 0004 loss_train: 1.8150 acc_train: 0.3345 loss_val: 1.8150 acc_val: 0.3345 time: 0.0376s
Epoch: 0005 loss_train: 1.4706 acc_train: 0.3311 loss_val: 1.4706 acc_val: 0.3311 time: 0.0361s
Epoch: 0006 loss_train: 1.1320 acc_train: 0.3513 loss_val: 1.1320 acc_val: 0.3513 time: 0.0371s
Epoch: 0007 loss_train: 1.1088 acc_train: 0.3463 loss_val: 1.

...


Num nodes 3001
         0         1  2
0        0  location  0
1        1      time  1
2        2  location  0
3        3      time  1
4        4     event  2
...    ...       ... ..
2996  2995     event  2
2997  2982  location  0
2998  2989  location  0
2999  2996  location  0
3000  3000  location  0

[3001 rows x 3 columns]
(3001, 3001) (3001, 512) 2373 628
Epoch: 0001 loss_train: 1.5838 acc_train: 0.3064 loss_val: 1.5838 acc_val: 0.3064 time: 0.0606s
Epoch: 0002 loss_train: 5.4149 acc_train: 0.3253 loss_val: 5.4149 acc_val: 0.3253 time: 0.0591s
Epoch: 0003 loss_train: 2.1769 acc_train: 0.3464 loss_val: 2.1769 acc_val: 0.3464 time: 0.0602s
Epoch: 0004 loss_train: 1.5455 acc_train: 0.3595 loss_val: 1.5455 acc_val: 0.3595 time: 0.0561s
Epoch: 0005 loss_train: 1.1177 acc_train: 0.3316 loss_val: 1.1177 acc_val: 0.3316 time: 0.0549s
Epoch: 0006 loss_train: 1.1046 acc_train: 0.3262 loss_val: 1.1046 acc_val: 0.3262 time: 0.0452s
Epoch: 0007 loss_train: 1.1190 acc_train: 0.3177 loss_val: 1.


Num nodes 1501
         0         1  2
0        0  location  0
1        1      time  1
2        2  location  0
3        3      time  1
4        7     event  2
...    ...       ... ..
1496  1493  location  0
1497  1494      time  1
1498  1489      time  1
1499  1499     event  2
1500  1497     event  2

[1501 rows x 3 columns]
(1501, 1501) (1501, 512) 1187 314
Epoch: 0001 loss_train: 1.6830 acc_train: 0.3184 loss_val: 1.6830 acc_val: 0.3184 time: 0.0410s
Epoch: 0002 loss_train: 5.8103 acc_train: 0.3243 loss_val: 5.8103 acc_val: 0.3243 time: 0.0369s
Epoch: 0003 loss_train: 1.4574 acc_train: 0.3142 loss_val: 1.4574 acc_val: 0.3142 time: 0.0380s
Epoch: 0004 loss_train: 1.8150 acc_train: 0.3345 loss_val: 1.8150 acc_val: 0.3345 time: 0.0370s
Epoch: 0005 loss_train: 1.4706 acc_train: 0.3311 loss_val: 1.4706 acc_val: 0.3311 time: 0.0407s
Epoch: 0006 loss_train: 1.1320 acc_train: 0.3513 loss_val: 1.1320 acc_val: 0.3513 time: 0.0394s
Epoch: 0007 loss_train: 1.1088 acc_train: 0.3463 loss_val: 1.


Num nodes 2001
         0         1  2
0        0  location  0
1        1      time  1
2        2  location  0
3        3      time  1
4        4     event  2
...    ...       ... ..
1996  1999      time  1
1997  1991      time  1
1998  1994      time  1
1999  1989     event  2
2000  1998  location  0

[2001 rows x 3 columns]
(2001, 2001) (2001, 512) 1583 418
Epoch: 0001 loss_train: 1.6103 acc_train: 0.3329 loss_val: 1.6103 acc_val: 0.3329 time: 0.0443s
Epoch: 0002 loss_train: 2.2680 acc_train: 0.3190 loss_val: 2.2680 acc_val: 0.3190 time: 0.0426s
Epoch: 0003 loss_train: 1.5783 acc_train: 0.3310 loss_val: 1.5783 acc_val: 0.3310 time: 0.0423s
Epoch: 0004 loss_train: 1.3940 acc_train: 0.3531 loss_val: 1.3940 acc_val: 0.3531 time: 0.0485s
Epoch: 0005 loss_train: 1.1473 acc_train: 0.3266 loss_val: 1.1473 acc_val: 0.3266 time: 0.0425s
Epoch: 0006 loss_train: 1.1218 acc_train: 0.3531 loss_val: 1.1218 acc_val: 0.3531 time: 0.0486s
Epoch: 0007 loss_train: 1.1223 acc_train: 0.3512 loss_val: 1.


Num nodes 3501
         0         1  2
0        0  location  0
1        1      time  1
2        2  location  0
3        4      time  1
4        5     event  2
...    ...       ... ..
3496  3480     event  2
3497  3495  location  0
3498  3484  location  0
3499  3491     event  2
3500  3498      time  1

[3501 rows x 3 columns]
(3501, 3501) (3501, 512) 2776 725
Epoch: 0001 loss_train: 1.6399 acc_train: 0.3206 loss_val: 1.6399 acc_val: 0.3206 time: 0.0698s
Epoch: 0002 loss_train: 4.3589 acc_train: 0.3271 loss_val: 4.3589 acc_val: 0.3271 time: 0.0752s
Epoch: 0003 loss_train: 1.8583 acc_train: 0.3577 loss_val: 1.8583 acc_val: 0.3577 time: 0.0620s
Epoch: 0004 loss_train: 1.2011 acc_train: 0.3354 loss_val: 1.2011 acc_val: 0.3354 time: 0.0511s
Epoch: 0005 loss_train: 1.1192 acc_train: 0.3379 loss_val: 1.1192 acc_val: 0.3379 time: 0.0515s
Epoch: 0006 loss_train: 1.1181 acc_train: 0.3213 loss_val: 1.1181 acc_val: 0.3213 time: 0.0543s
Epoch: 0007 loss_train: 1.1188 acc_train: 0.3332 loss_val: 1.


Num nodes 2501
         0         1  2
0        0  location  0
1        1      time  1
2        2  location  0
3        3      time  1
4        4     event  2
...    ...       ... ..
2496  2492     event  2
2497  2481     event  2
2498  2497     event  2
2499  2500  location  0
2500  2498  location  0

[2501 rows x 3 columns]
(2501, 2501) (2501, 512) 1983 518
Epoch: 0001 loss_train: 1.5512 acc_train: 0.3232 loss_val: 1.5512 acc_val: 0.3232 time: 0.0538s
Epoch: 0002 loss_train: 4.8815 acc_train: 0.3243 loss_val: 4.8815 acc_val: 0.3243 time: 0.0535s
Epoch: 0003 loss_train: 1.3525 acc_train: 0.3323 loss_val: 1.3525 acc_val: 0.3323 time: 0.0570s
Epoch: 0004 loss_train: 1.3430 acc_train: 0.3232 loss_val: 1.3430 acc_val: 0.3232 time: 0.0472s
Epoch: 0005 loss_train: 1.3213 acc_train: 0.3384 loss_val: 1.3213 acc_val: 0.3384 time: 0.0445s
Epoch: 0006 loss_train: 1.1853 acc_train: 0.3449 loss_val: 1.1853 acc_val: 0.3449 time: 0.0420s
Epoch: 0007 loss_train: 1.1163 acc_train: 0.3253 loss_val: 1.

In [ ]:
!zip results.zip *.csv

updating: gat_results_r1_news_cluster_5w1h_graph_hin.csv (deflated 92%)
updating: gat_results_r2_news_cluster_5w1h_graph_hin.csv (deflated 92%)
updating: gat_results_r3_news_cluster_5w1h_graph_hin.csv (deflated 92%)
updating: gat_results_r4_news_cluster_5w1h_graph_hin.csv (deflated 92%)
updating: gat_results_r5_news_cluster_5w1h_graph_hin.csv (deflated 92%)


In [ ]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/results.zip')

/kaggle/working/results.zip

<a href="/output/kaggle/working/pyGAT/gat_results_r1497_news_cluster_5w1h_graph_hin.csv"> Download File </a>

In [ ]:
os.chdir('/kaggle/working')

In [ ]:
!ls /kaggle/working/

In [ ]:
!mv /kaggle/working/pyGAT/results.zip /

In [ ]:
G = nx.read_edgelist(path_datasets+"GraphRNN_RNN_EVENT_4_128_pred_10_1_4000_nodes.edge_list")